In [ ]:
# First clone inside Datasets Section inside Kaggle
# https://github.com/VarunGumma/IndicTransToolkit/tree/main/IndicTransToolkit

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indictranstoolkit/IndicTransToolkit-main/LICENSE
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/.gitignore
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/pyproject.toml
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/README.md
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/requirements.txt
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/CHANGELOG.md
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/setup.py
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/IndicTransToolkit/processor.cpython-310-x86_64-linux-gnu.so
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/IndicTransToolkit/version.txt
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/IndicTransToolkit/version.py
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/IndicTransToolkit/processor.pyx
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/IndicTransToolkit/collator.py
/kaggle/input/indictranstoolkit/IndicTransToolkit-main/IndicTransToolkit/pro

In [2]:

!cp -r /kaggle/input/indictranstoolkit/IndicTransToolkit-main /kaggle/working/IndicTransToolkit
%cd /kaggle/working/IndicTransToolkit
!pip install -e .
%cd /kaggle/working
import sys
sys.path.append('/kaggle/working/IndicTransToolkit')
from IndicTransToolkit import evaluator, collator
from IndicTransToolkit import IndicProcessor


/kaggle/working/IndicTransToolkit
Obtaining file:///kaggle/working/IndicTransToolkit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library.git to /tmp/pip-install-avvxd40v/indic-nlp-library-it2_a610d10e06d64d2f992e08d307cf1d22
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library.git /tmp/pip-install-avvxd40v/indic-nlp-library-it2_a610d10e06d64d2f992e08d307cf1d22
  Resolved https://github.com/VarunGumma/indic_nlp_library.git to commit 342a7e95735e88949528bdca371518cb0ba5bb5d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB

In [6]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit import IndicProcessor
from tqdm import tqdm

# ==== DEVICE CONFIG ====
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ==== LANGUAGE CONFIG ====
src_lang, tgt_lang = "hin_Deva", "mar_Deva"  # Set target lang if needed

# ==== MODEL & TOKENIZER ====
model_name = "ai4bharat/indictrans2-indic-indic-dist-320M"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

# ==== INPUT FILE CONFIG ====
input_file = "/kaggle/input/sentiment-dataset/training-data.txt"
output_file = "/kaggle/working/training-data-marathi.txt"

# ==== READ INPUT ====
with open(input_file, "r", encoding="utf-8") as f:
    input_sentences = [line.strip() for line in f if line.strip()]

# ==== TRANSLATE IN BATCHES ====
translations = []
batch_size = 4  # Increase if memory allows

for i in tqdm(range(0, len(input_sentences), batch_size), desc="Translating"):
    input_batch = input_sentences[i:i+batch_size]

    # Preprocess
    batch = ip.preprocess_batch(
        input_batch,
        src_lang=src_lang,
        tgt_lang=tgt_lang,
    )

    # Tokenize
    inputs = tokenizer(
        batch,
        truncation=True,
        padding="longest",
        return_tensors="pt",
        return_attention_mask=True,
    ).to(DEVICE)

    # Generate
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            use_cache=True,
            min_length=0,
            max_length=256,
            num_beams=5,
            num_return_sequences=1,
        )

    # Decode
    with tokenizer.as_target_tokenizer():
        decoded = tokenizer.batch_decode(
            generated_tokens.detach().cpu().tolist(),
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

    # Postprocess
    translations.extend(ip.postprocess_batch(decoded, lang=tgt_lang))

    # Free memory
    del inputs, generated_tokens
    torch.cuda.empty_cache()

# ==== WRITE OUTPUT ====
with open(output_file, "w", encoding="utf-8") as f:
    for input_sentence, translation in zip(input_sentences, translations):
        # f.write(f"{src_lang}: {input_sentence}\n")
        f.write(f"{translation}\n")

print(f"✅ Translations written to: {output_file}")


Translating: 100%|██████████| 451/451 [06:23<00:00,  1.18it/s]

✅ Translations written to: /kaggle/working/training-data-marathi.txt


In [10]:
# extracting labels from training data 
input_path = "/kaggle/input/translated-datasets/training-data-telugu.txt"  # <-- adjust this to your dataset
output_path = "/kaggle/working/training-data-telugu-labels.txt"

# Read the data
with open(input_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file if line.strip()]

# Extract labels (assuming last word in each line is the label)
labels = [line.split()[-1] for line in lines]

# Save labels to output file
with open(output_path, 'w', encoding='utf-8') as out_file:
    for label in labels:
        out_file.write(label + '\n')

print(f"✅ Labels extracted and saved to: {output_path}")


✅ Labels extracted and saved to: /kaggle/working/training-data-telugu-labels.txt


In [12]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from pickle import dump
from sklearn.linear_model import LogisticRegression, SGDClassifier
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from random import shuffle


def readLinesFromFile(filePath):
    with open(filePath, 'r', encoding='utf-8') as fileRead:
        return [line.strip() for line in fileRead.readlines() if line.strip()]


def createTFIDFVectorsFromTrainData(trainData, analyzer='word', ngram_range=(1, 1)):
    tfIDFVect = TfidfVectorizer(analyzer=analyzer, ngram_range=ngram_range)
    trainTFIdf = tfIDFVect.fit_transform(trainData)
    return trainTFIdf, tfIDFVect


def dumpObjectIntoFile(filePath, dataObject):
    with open(filePath, 'wb') as fileDump:
        dump(dataObject, fileDump)


def fitTrainDataWithClassifier(clf, trainData, trainLabels):
    clf.fit(trainData, trainLabels)
    return clf


def SVMClassifier(trainData, trainLabels):
    svm = LinearSVC()
    return fitTrainDataWithClassifier(svm, trainData, trainLabels)


def logisticClassifier(trainData, trainLabels):
    logit = LogisticRegression()
    return fitTrainDataWithClassifier(logit, trainData, trainLabels)


def gradientDescent(trainData, trainLabels):
    sgd = SGDClassifier(loss='perceptron')
    return fitTrainDataWithClassifier(sgd, trainData, trainLabels)


def gradientBoost(trainData, trainLabels):
    gradBoost = GradientBoostingClassifier()
    return fitTrainDataWithClassifier(gradBoost, trainData, trainLabels)


def multinomialNBClassifier(trainData, trainLabels):
    mNB = MultinomialNB(alpha=0.1)
    return fitTrainDataWithClassifier(mNB, trainData, trainLabels)


def main():
    # ✅ Direct file paths (Kaggle)
    dataFilePath = '/kaggle/input/sentiment-dataset/training-data.txt'
    labelFilePath = '/kaggle/input/labels-datasets/training-data-labels.txt'
    classifier = 'svm'  # 👈 You can change this to 'logistic', 'multi-nb', 'sgd', etc.

    char_analyzer = 'char'
    char_ngram_range = (2, 5)
    word_analyzer = 'word'
    word_ngram_range = (1, 1)

    trainData = readLinesFromFile(dataFilePath)
    trainLabels = readLinesFromFile(labelFilePath)

    assert len(trainLabels) == len(trainData)

    indexes = list(range(len(trainLabels)))
    shuffle(indexes)
    trainData = [trainData[i] for i in indexes]
    trainLabels = [trainLabels[i] for i in indexes]

    wordTrainTFIdf, wordTfIdfVect = createTFIDFVectorsFromTrainData(trainData, word_analyzer, word_ngram_range)
    charTrainTFIdf, charTfIdfVect = createTFIDFVectorsFromTrainData(trainData, char_analyzer, char_ngram_range)
    combinedTrainTfIdf = hstack([wordTrainTFIdf, charTrainTFIdf])

    if re.search('svm', classifier, re.I):
        classifierToSelect = SVMClassifier(combinedTrainTfIdf, trainLabels)
    elif re.search('logistic', classifier, re.I):
        classifierToSelect = logisticClassifier(combinedTrainTfIdf, trainLabels)
    elif re.search('multi-nb', classifier, re.I):
        classifierToSelect = multinomialNBClassifier(combinedTrainTfIdf, trainLabels)
    elif re.search('sgd', classifier, re.I):
        classifierToSelect = gradientDescent(combinedTrainTfIdf, trainLabels)
    elif re.search('gradient-boosting', classifier, re.I):
        classifierToSelect = gradientBoost(combinedTrainTfIdf, trainLabels)
    else:
        raise ValueError("Unsupported classifier selected.")

    # ✅ Save outputs to /kaggle/working/
    dumpObjectIntoFile(f'/kaggle/working/train-vect-word-{classifier}.pkl', wordTfIdfVect)
    dumpObjectIntoFile(f'/kaggle/working/train-vect-char-{classifier}.pkl', charTfIdfVect)
    dumpObjectIntoFile(f'/kaggle/working/classifier-{classifier}.pkl', classifierToSelect)

    print("✅ Model and vectorizers saved to /kaggle/working/")


# ✅ Run it
main()


✅ Model and vectorizers saved to /kaggle/working/


In [15]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from pickle import load

def readLinesFromFile(filePath):
    with open(filePath, 'r', encoding='utf-8') as fileRead:
        return [line.strip() for line in fileRead.readlines() if line.strip()]

def createTestTfIdf(testData, tfIdfVect):
    return tfIdfVect.transform(testData)

def loadObjectFromFile(filePath):
    with open(filePath, 'rb') as fileLoad:
        return load(fileLoad)

def writeListToFile(filePath, dataList):
    with open(filePath, 'w', encoding='utf-8') as fileWrite:
        fileWrite.write('\n'.join(dataList))

def predictOnFeatures(classifier, features):
    return classifier.predict(features)

# 🔽 Paths
testFilePath = '/kaggle/input/testsss/unlabeled_reviews.txt'
wordVectPath = '/kaggle/working/train-vect-word-svm.pkl'
charVectPath = '/kaggle/working/train-vect-char-svm.pkl'
classifierPath = '/kaggle/working/classifier-svm.pkl'
outputPredPath = '/kaggle/working/test-predictions.txt'

# 🔄 Prediction steps
testData = readLinesFromFile(testFilePath)
wordVect = loadObjectFromFile(wordVectPath)
charVect = loadObjectFromFile(charVectPath)
clf = loadObjectFromFile(classifierPath)

wordTestTfIdf = createTestTfIdf(testData, wordVect)
charTestTfIdf = createTestTfIdf(testData, charVect)
combinedTestTfIdf = hstack([wordTestTfIdf, charTestTfIdf])

predictions = predictOnFeatures(clf, combinedTestTfIdf)

writeListToFile(outputPredPath, predictions)

print("✅ Predictions saved to:", outputPredPath)


✅ Predictions saved to: /kaggle/working/test-predictions.txt
